In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="your model Name")

default_system_prompt = (
    "당신은 사용자의 요청에만 충실하게 답변하는 사실 기반의 중립적인 AI 어시스턴트입니다."
)


In [ ]:
messages = [
    SystemMessage(content=default_system_prompt),
    HumanMessage(content="질문 쓰는 곳")
]

response = llm.invoke(messages)
print(response.content)

---

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
import os

llm = ChatOllama(model="your model Name")

def load_and_split_documents(loaders):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

    all_splits = []
    
    for loader in loaders:
        docs = loader.load_and_split()
        splits = text_splitter.split_documents(docs)
        all_splits.extend(splits)

    return all_splits

def get_loaders_from_directory(folder_path):
    loaders = []

    for filename in os.listdir(folder_path):
        full_path = os.path.join(folder_path, filename)

        if filename.endswith(".pdf"):
            loaders.append(PyPDFLoader(full_path))
        elif filename.endswith(".txt"):
            loaders.append(TextLoader(full_path, encoding="utf-8"))
        # elif filename.endswith(".docx"):
        #     loaders.append(Docx2txtLoader(full_path))
        # elif filename.endswith(".md"):
        #     loaders.append(MarkdownLoader(full_path))  # 필요시 확장 가능

    return loaders

#모든 경로에 파일 다읽어오게게
loaders = get_loaders_from_directory('your File Path')

documents = load_and_split_documents(loaders)

model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma.from_documents(documents = documents , embedding = embedding_model)

custom_template = """당신은 사용자의 요청에만 충실하게 답변하는 사실 기반의 중립적인 AI 어시스턴트입니다.

[문서 정보]
{context}

[사용자 질문]
{question}

[응답]
"""
QA_PROMPT = PromptTemplate(template=custom_template, input_variables=["context", "question"])

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer" 
)

# 체인 구성
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT},
    return_source_documents=True,
    output_key="answer"  
)

def AskLLM(ask : str):
    response = chain.invoke({"question": f"{ask}"})
    print_response_summary(response=response, memory=memory, max_docs=1, max_history=4)
    return response

def print_response_summary(response, memory=None, max_docs=1, max_history=4):

    print(" [AI 응답] :", response.get('answer', '[응답 없음]'))

    source_docs = response.get("source_documents", [])[:max_docs]
    if source_docs:
        doc = source_docs[0]
        print("\n[문서 1번]")
        print("내용 (요약) :", doc.page_content[:50])
        print("-[출처 메타데이터] : ", doc.metadata)
    else:
        print("\n 참고 문서 없음")

    if memory:
        print("\n [최근 대화 히스토리]")
        # 최근 N개의 메시지만 출력
        messages = memory.chat_memory.messages[-max_history:]
        for msg in messages:
            role = "    [사용자]" if msg.type == "human" else "    [AI]"
            print(f"{role}: {msg.content}")
                             
# # 응답 출력
# print("응답:", response['answer'])

# # 참고 문서 출력
# source_docs = response.get("source_documents", [])
# for i, doc in enumerate(source_docs):
#     print(f"\n문서 {i+1}:")
#     print("내용 (요약):", doc.page_content[:300])
#     print("출처 메타데이터:", doc.metadata)

# for msg in memory.chat_memory.messages:
#     role = " 사용자" if msg.type == "human" else " AI"
#     print(f"{role}: {msg.content}")

In [ ]:
response = AskLLM("이 문서의 핵심은?")